The code ends up with insufficient memory

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.4 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
# Load the pretrained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
# Load the restaurant review dataset
restaurant_df = pd.read_csv('/content/Restaurant_Reviews.csv')
restaurant_df.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [ ]:
# Prepare your data for classification
restaurant_texts = restaurant_df['Review'].tolist()
restaurant_labels = restaurant_df['Liked'].tolist()

In [ ]:
restaurant_encodings = tokenizer(restaurant_texts, padding=True, truncation=True, return_tensors='pt')
restaurant_labels = torch.tensor(restaurant_labels[:len(restaurant_encodings['input_ids'])]).clone().detach()

In [ ]:
print(len(restaurant_texts))
print(len(restaurant_encodings['input_ids']))
print(len(restaurant_labels))

1000
1000
1000


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into train and test sets
restaurant_train_texts, restaurant_test_texts, restaurant_train_labels, restaurant_test_labels = \
    train_test_split(restaurant_texts, restaurant_labels, test_size=0.2, random_state=42)

# Tokenize the train and test texts and convert them to tensors
restaurant_train_encodings = tokenizer(restaurant_train_texts, padding=True, truncation=True, return_tensors='pt')
restaurant_test_encodings = tokenizer(restaurant_test_texts, padding=True, truncation=True, return_tensors='pt')

# Convert train and test labels to tensors
restaurant_train_labels = torch.tensor(restaurant_train_labels).clone().detach()
restaurant_test_labels = torch.tensor(restaurant_test_labels).clone().detach()


<ipython-input-8-d2736dadec9d>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  restaurant_train_labels = torch.tensor(restaurant_train_labels).clone().detach()
<ipython-input-8-d2736dadec9d>:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  restaurant_test_labels = torch.tensor(restaurant_test_labels).clone().detach()


In [ ]:
# Fine-tune the BERT model on the restaurant review data
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
model.train()
for epoch in range(3):  # Adjust the number of training epochs as needed
    optimizer.zero_grad()
    outputs = model(**restaurant_train_encodings, labels=restaurant_train_labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()

Your session crashed after using all available RAM. If you are interested in access to high-RAM runtimes, you may want to check out Colab Pro.